In [1]:
import oneflow as flow 
import oneflow.typing as tp

In [2]:
BATCH_SIZE = 100

In [3]:
@flow.global_function(type="train")
#作业函数
def train_job(
    images: tp.Numpy.Placeholder((BATCH_SIZE, 1, 28, 28), dtype=flow.float),
    labels: tp.Numpy.Placeholder((BATCH_SIZE,), dtype=flow.int32),
) -> tp.Numpy:
    with flow.scope.placement("cpu", "0:0"):
        initializer = flow.truncated_normal(0.1)
        reshape = flow.reshape(images, [images.shape[0], -1])
        hidden = flow.layers.dense(
            reshape,
            512,
            activation=flow.nn.relu,
            kernel_initializer=initializer,
            name="dense1",
        )
        logits = flow.layers.dense(
            hidden, 10, kernel_initializer=initializer, name="dense2"
        )
        loss = flow.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)

    lr_scheduler = flow.optimizer.PiecewiseConstantScheduler([], [0.1])
    flow.optimizer.SGD(lr_scheduler, momentum=0).minimize(loss)

    return loss


In [4]:
#初始化checkpoint
check_point = flow.train.CheckPoint()
check_point.init()

In [5]:
 (train_images, train_labels), (test_images, test_labels) = flow.data.load_mnist(
        BATCH_SIZE, BATCH_SIZE
    )

File mnist.npz already exist, path: ./mnist.npz


In [6]:
for i, (images, labels) in enumerate(zip(train_images, train_labels)):
    loss = train_job(images, labels)
    if i % 20 == 0:
        print(loss.mean())

3.7168775
0.8495066
0.5526892
0.36495122
0.36913228
0.41484782
0.4176693
0.48654634
0.5082862
0.32437402
0.5002268
0.24401057
0.31899843
0.25835595
0.17804009
0.36032844
0.34623224
0.28273284
0.3515799
0.29099074
0.27752814
0.3082276
0.18332332
0.43328112
0.2645805
0.3003138
0.33684796
0.29110742
0.17175771
0.22444227
